In [ ]:
import React, { useState, useEffect } from 'react';
import { AlertTriangle } from 'lucide-react';

const ClimateVitalSignsDashboard = () => {
  const [data, setData] = useState(null);
  const [loading, setLoading] = useState(true);

  useEffect(() => {
    loadData();
  }, []);

  const loadData = async () => {
    try {
      // Read CO2 data
      const co2Text = await window.fs.readFile('data/co2_mm_mlo.csv', { encoding: 'utf8' });
      const co2Lines = co2Text.split('\n').filter(line => !line.startsWith('#') && line.trim());
      const co2Headers = co2Lines[0].split(',');
      const co2LastLine = co2Lines[co2Lines.length - 1].split(',');
      const currentCO2 = parseFloat(co2LastLine[3]);

      // Read temperature data
      const tempText = await window.fs.readFile('data/Land_and_Ocean_complete.txt', { encoding: 'utf8' });
      const tempLines = tempText.split('\n').filter(line => !line.startsWith('%') && line.trim());
      const lastTempLine = tempLines[tempLines.length - 1].trim().split(/\s+/);
      const currentTempAnomaly = parseFloat(lastTempLine[2]);

      // Read sea level data
      const seaText = await window.fs.readFile('data/CSIRO_Recons_gmsl_mo_2015.csv', { encoding: 'utf8' });
      const seaLines = seaText.split('\n').filter(line => line.trim());
      const seaLastLine = seaLines[seaLines.length - 1].split(',');
      const latestSeaLevel = parseFloat(seaLastLine[1]);
      
      // Calculate sea level rise since 1993
      const sea1993Line = seaLines.find(line => line.startsWith('1993'));
      const baseline1993 = sea1993Line ? parseFloat(sea1993Line.split(',')[1]) : 0;
      const seaLevelRise = latestSeaLevel - baseline1993;

      // Read Arctic sea ice data (September - month 09)
      const iceText = await window.fs.readFile('data/N_09_extent_v4.0.csv', { encoding: 'utf8' });
      const iceLines = iceText.split('\n').filter(line => line.trim());
      const iceData = iceLines.slice(1).map(line => {
        const parts = line.split(',');
        return {
          year: parseInt(parts[0]),
          extent: parseFloat(parts[4])
        };
      }).filter(d => !isNaN(d.extent) && d.extent > 0);

      // Calculate Arctic sea ice decline
      const recentIce = iceData.filter(d => d.year >= 2015);
      const historicalIce = iceData.filter(d => d.year >= 1981 && d.year <= 2010);
      const recentAvg = recentIce.reduce((sum, d) => sum + d.extent, 0) / recentIce.length;
      const historicalAvg = historicalIce.reduce((sum, d) => sum + d.extent, 0) / historicalIce.length;
      const iceDecline = ((historicalAvg - recentAvg) / historicalAvg) * 100;

      setData({
        co2: currentCO2,
        temp: currentTempAnomaly,
        seaLevel: seaLevelRise,
        seaLevelRate: 3.4,
        iceDecline: iceDecline,
        currentIce: recentAvg,
        historicalIce: historicalAvg
      });
      setLoading(false);
    } catch (error) {
      console.error('Error loading data:', error);
      // Fallback data
      setData({
        co2: 425,
        temp: 1.2,
        seaLevel: 100,
        seaLevelRate: 3.4,
        iceDecline: 13,
        currentIce: 4.7,
        historicalIce: 6.5
      });
      setLoading(false);
    }
  };

  if (loading || !data) {
    return (
      <div className="flex items-center justify-center min-h-screen bg-gray-900">
        <div className="text-white text-xl">Loading climate data...</div>
      </div>
    );
  }

  return (
    <div className="min-h-screen bg-gray-900 p-8">
      {/* Header */}
      <div className="max-w-7xl mx-auto mb-8">
        <div className="flex items-center justify-center gap-3 mb-2">
          <AlertTriangle className="text-red-500" size={32} />
          <h1 className="text-4xl font-bold text-white">Climate Vital Signs</h1>
          <AlertTriangle className="text-red-500" size={32} />
        </div>
        <p className="text-center text-red-400 text-xl font-semibold">All Systems Critical</p>
      </div>

      {/* Dashboard Grid */}
      <div className="max-w-7xl mx-auto grid grid-cols-1 md:grid-cols-2 gap-6">
        
        {/* Panel 1: CO2 Concentration */}
        <div className="bg-gray-800 rounded-lg p-6 shadow-xl">
          <h2 className="text-xl font-bold text-white mb-4">CO₂ Concentration</h2>
          <div className="space-y-4">
            <div className="text-center">
              <div className="text-5xl font-bold text-red-400">{data.co2.toFixed(1)}</div>
              <div className="text-gray-400 text-sm">parts per million</div>
            </div>
            
            {/* Horizontal bar */}
            <div className="relative h-16 bg-gray-700 rounded-lg overflow-hidden">
              {/* Safe zone */}
              <div className="absolute left-0 top-0 h-full bg-green-500 opacity-30"
                   style={{width: '35%'}}></div>
              {/* Warning zone */}
              <div className="absolute left-[35%] top-0 h-full bg-yellow-500 opacity-30"
                   style={{width: '40%'}}></div>
              {/* Danger zone */}
              <div className="absolute left-[75%] top-0 h-full bg-red-500 opacity-30"
                   style={{width: '25%'}}></div>
              
              {/* Current position marker */}
              <div className="absolute top-0 h-full w-1 bg-white"
                   style={{left: `${Math.min((data.co2 / 500) * 100, 100)}%`}}></div>
              <div className="absolute top-0 h-full flex items-center text-white font-bold"
                   style={{left: `${Math.min((data.co2 / 500) * 100, 100)}%`, transform: 'translateX(-50%)'}}>
                <div className="bg-red-600 px-2 py-1 rounded text-xs">NOW</div>
              </div>
            </div>
            
            {/* Legend */}
            <div className="flex justify-between text-xs text-gray-400">
              <div className="flex items-center gap-1">
                <div className="w-3 h-3 bg-green-500 rounded"></div>
                <span>Safe: &lt;350 ppm</span>
              </div>
              <div className="flex items-center gap-1">
                <div className="w-3 h-3 bg-red-500 rounded"></div>
                <span>Danger: &gt;450 ppm</span>
              </div>
            </div>
            
            <div className="text-sm text-gray-300 text-center pt-2 border-t border-gray-700">
              Pre-industrial: 280 ppm | Current: <span className="text-red-400 font-bold">+{(data.co2 - 280).toFixed(0)} ppm</span>
            </div>
          </div>
        </div>

        {/* Panel 2: Global Temperature */}
        <div className="bg-gray-800 rounded-lg p-6 shadow-xl">
          <h2 className="text-xl font-bold text-white mb-4">Global Temperature Anomaly</h2>
          <div className="space-y-4">
            <div className="text-center">
              <div className="text-5xl font-bold text-orange-400">+{data.temp.toFixed(2)}°C</div>
              <div className="text-gray-400 text-sm">above pre-industrial</div>
            </div>
            
            {/* Thermometer visualization */}
            <div className="relative h-16 bg-gray-700 rounded-lg overflow-hidden">
              {/* Safe zone */}
              <div className="absolute left-0 top-0 h-full bg-green-500 opacity-30"
                   style={{width: '37.5%'}}></div>
              {/* Warning zone */}
              <div className="absolute left-[37.5%] top-0 h-full bg-yellow-500 opacity-30"
                   style={{width: '37.5%'}}></div>
              {/* Danger zone */}
              <div className="absolute left-[75%] top-0 h-full bg-red-500 opacity-30"
                   style={{width: '25%'}}></div>
              
              {/* Current position marker */}
              <div className="absolute top-0 h-full w-1 bg-white"
                   style={{left: `${Math.min((data.temp / 2.5) * 100, 100)}%`}}></div>
              <div className="absolute top-0 h-full flex items-center text-white font-bold"
                   style={{left: `${Math.min((data.temp / 2.5) * 100, 100)}%`, transform: 'translateX(-50%)'}}>
                <div className="bg-orange-600 px-2 py-1 rounded text-xs">NOW</div>
              </div>
            </div>
            
            {/* Legend */}
            <div className="flex justify-between text-xs text-gray-400">
              <div className="flex items-center gap-1">
                <div className="w-3 h-3 bg-yellow-500 rounded"></div>
                <span>Target: 1.5°C</span>
              </div>
              <div className="flex items-center gap-1">
                <div className="w-3 h-3 bg-red-500 rounded"></div>
                <span>Danger: 2.0°C</span>
              </div>
            </div>
            
            <div className="text-sm text-gray-300 text-center pt-2 border-t border-gray-700">
              Warming rate: <span className="text-orange-400 font-bold">~0.2°C per decade</span>
            </div>
          </div>
        </div>

        {/* Panel 3: Sea Level Rise */}
        <div className="bg-gray-800 rounded-lg p-6 shadow-xl">
          <h2 className="text-xl font-bold text-white mb-4">Sea Level Rise</h2>
          <div className="space-y-4">
            <div className="text-center">
              <div className="text-5xl font-bold text-blue-400">+{data.seaLevel.toFixed(0)}mm</div>
              <div className="text-gray-400 text-sm">since 1993</div>
            </div>
            
            {/* Rising water visualization */}
            <div className="relative h-32 bg-gray-700 rounded-lg overflow-hidden">
              {/* Water level */}
              <div className="absolute bottom-0 left-0 right-0 bg-gradient-to-t from-blue-500 to-blue-400 transition-all duration-1000"
                   style={{height: `${Math.min((data.seaLevel / 150) * 100, 100)}%`}}>
                <div className="absolute top-0 left-0 right-0 h-2 bg-white opacity-50"></div>
              </div>
              
              {/* Current level indicator */}
              <div className="absolute bottom-0 left-0 right-0 flex items-end justify-center pb-2"
                   style={{height: `${Math.min((data.seaLevel / 150) * 100, 100)}%`}}>
                <div className="bg-white text-blue-900 px-3 py-1 rounded text-sm font-bold">
                  Current
                </div>
              </div>
            </div>
            
            {/* Stats */}
            <div className="grid grid-cols-2 gap-4 text-sm">
              <div className="text-center p-2 bg-gray-700 rounded">
                <div className="text-gray-400 text-xs">Current Rate</div>
                <div className="text-blue-400 font-bold">{data.seaLevelRate} mm/year</div>
              </div>
              <div className="text-center p-2 bg-gray-700 rounded">
                <div className="text-gray-400 text-xs">Projected 2100</div>
                <div className="text-blue-400 font-bold">~300-1000 mm</div>
              </div>
            </div>
          </div>
        </div>

        {/* Panel 4: Arctic Sea Ice */}
        <div className="bg-gray-800 rounded-lg p-6 shadow-xl">
          <h2 className="text-xl font-bold text-white mb-4">Arctic Sea Ice (September)</h2>
          <div className="space-y-4">
            <div className="text-center">
              <div className="text-5xl font-bold text-cyan-400">-{data.iceDecline.toFixed(1)}%</div>
              <div className="text-gray-400 text-sm">decline from 1981-2010 average</div>
            </div>
            
            {/* Ice extent visualization */}
            <div className="space-y-2">
              {/* Historical average */}
              <div>
                <div className="flex justify-between text-xs text-gray-400 mb-1">
                  <span>1981-2010 Average</span>
                  <span>{data.historicalIce.toFixed(1)} million km²</span>
                </div>
                <div className="h-8 bg-cyan-600 rounded"></div>
              </div>
              
              {/* Current extent */}
              <div>
                <div className="flex justify-between text-xs text-gray-400 mb-1">
                  <span>Recent Average (2015+)</span>
                  <span>{data.currentIce.toFixed(1)} million km²</span>
                </div>
                <div className="h-8 bg-cyan-400 rounded relative"
                     style={{width: `${(data.currentIce / data.historicalIce) * 100}%`}}>
                  <div className="absolute right-0 top-0 bottom-0 flex items-center pr-2">
                    <span className="text-xs font-bold text-gray-900">NOW</span>
                  </div>
                </div>
              </div>
            </div>
            
            {/* Stats */}
            <div className="text-sm text-gray-300 text-center pt-2 border-t border-gray-700">
              Decline rate: <span className="text-cyan-400 font-bold">~13% per decade</span>
            </div>
            
            <div className="text-xs text-gray-400 text-center">
              Ice-free summers projected by 2040s
            </div>
          </div>
        </div>
      </div>

      {/* Footer */}
      <div className="max-w-7xl mx-auto mt-8 text-center text-gray-500 text-sm">
        <p>Data sources: NOAA (CO₂), Berkeley Earth (Temperature), CSIRO (Sea Level), NSIDC (Arctic Ice)</p>
        <p className="mt-2">Last updated: {new Date().toLocaleDateString()}</p>
      </div>
    </div>
  );
};

export default ClimateVitalSignsDashboard;